In [1]:
import pandas as pd
import numpy as np
import csv
import tempfile
import os
import requests
from PIL import Image
from PIL import UnidentifiedImageError
import io
import base64
import geopandas as gpd
from shapely.geometry import Point
from arcgis.features import GeoAccessor
from arcgis.gis import GIS
from arcgis import GIS
from arcgis.features import Feature, FeatureSet, Table
from arcgis.features import FeatureLayer
from arcgis.features import FeatureLayerCollection
from arcgis import features

In [13]:
df2 = pd.read_csv('photos_data.csv')

In [10]:
df = pd.read_csv('attributes_states.csv')

In [11]:
# Create a mapping dictionary using 'photo_id' as keys and 'photo_src' as values
mapping_dict = df2.set_index('photo_id')['photo_src'].to_dict()

# Map the 'photo_src' values to 'df' based on 'PhotoID' and 'photo_id'
df['photo_src'] = df['PhotoID'].map(mapping_dict)
df.head()

,PhotoID,Title,Description,URL,Latitude,Longitude,AlbumID,AlbumTitle,Identifier,MissionDescription,ProjectType,Partner,County,State,photo_src
0,51604730510,TAuch_Infrastructure-Rover_Pipeline_Repair-Ene...,"Photo citation: Ted Auch, FracTracker Alliance...",https://www.flickr.com/photos/fractracker/5160...,39.868655,-80.833423,72157715916543893,Appalachian Buildout,TAuch,Infrastructure-Rover,Rover_Pipeline_Repair,EnergyTransferPartner,BelmontCounty,OH,https://live.staticflickr.com/65535/5160473051...
1,51604505439,TAuch_Infrastructure-Rover_Pipeline_Repair-Ene...,"Photo citation: Ted Auch, FracTracker Alliance...",https://www.flickr.com/photos/fractracker/5160...,39.868177,-80.832106,72157715916543893,Appalachian Buildout,TAuch,Infrastructure-Rover,Rover_Pipeline_Repair,EnergyTransferPartner,BelmontCounty,OH,https://live.staticflickr.com/65535/5160450543...
2,51603848491,TAuch_Infrastructure-Rover_Pipeline_Repair-Ene...,"Photo citation: Ted Auch, FracTracker Alliance...",https://www.flickr.com/photos/fractracker/5160...,39.868111,-80.832081,72157715916543893,Appalachian Buildout,TAuch,Infrastructure-Rover,Rover_Pipeline_Repair,EnergyTransferPartner,BelmontCounty,OH,https://live.staticflickr.com/65535/5160384849...
3,51603848526,TAuch_Infrastructure-Rover_Pipeline_Repair-Ene...,"Photo citation: Ted Auch, FracTracker Alliance...",https://www.flickr.com/photos/fractracker/5160...,39.868288,-80.830231,72157715916543893,Appalachian Buildout,TAuch,Infrastructure-Rover,Rover_Pipeline_Repair,EnergyTransferPartner,BelmontCounty,OH,https://live.staticflickr.com/65535/5160384852...
4,51604730665,TAuch_Infrastructure-Rover_Pipeline_Repair-Ene...,"Photo citation: Ted Auch, FracTracker Alliance...",https://www.flickr.com/photos/fractracker/5160...,39.864536,-80.832995,72157715916543893,Appalachian Buildout,TAuch,Infrastructure-Rover,Rover_Pipeline_Repair,EnergyTransferPartner,BelmontCounty,OH,https://live.staticflickr.com/65535/5160473066...


In [14]:
df.to_csv('attributes_updated.csv', index=False)

In [15]:
#connect to arcGIS account
gis = GIS("https://www.arcgis.com", "willemluyten", "Willembug17!")

In [16]:
#create a feature set to store the data from attribute table:
feature_set = FeatureSet(features=[])
feature_set.spatial_reference = {"wkid": 4326}

In [17]:
#put attribute table into feature set
with open('attributes_updated.csv', 'r') as file:
    reader = csv.DictReader(file)
    for row in reader:
        #check if latitude and longitude values are non-empty
        latitude = float(row['Latitude']) if row['Latitude'] else None
        longitude = float(row['Longitude']) if row['Longitude'] else None

        #proceed only if lon and lat exist
        if latitude is not None and longitude is not None:
            feature = Feature(
                geometry={
                    "x": longitude,
                    "y": latitude,
                    "spatialReference": feature_set.spatial_reference
                },
                attributes=row
            )
            feature_set.features.append(feature)

# publish the feature set as a hosted feature layer

In [23]:
import uuid

# Generate a unique identifier
unique_id = str(uuid.uuid4())[:8]

In [24]:
#empty lists to store geometry and attributes
geometries = []
attributes = []

#extract lon/lat and attributes from feature set
for feature in feature_set.features:
    x = feature.geometry['x']
    y = feature.geometry['y']
    geometry = Point(x, y)
    geometries.append(geometry)
    attributes.append(feature.attributes)

#create a geoDF
gdf = gpd.GeoDataFrame(attributes, geometry=geometries)

#save the GeoDF to a GeoJSON file
#output_geojson = "/Users/willemluyten/Desktop/output.geojson"
output_geojson = f"/Users/willemluyten/Desktop/output_{unique_id}.geojson"
gdf.to_file(output_geojson, driver='GeoJSON')


#add the GeoJSON file as an item
item_properties = {
    'type': 'GeoJson',
    'title': 'FracTracker Flickr Map 3',
    'tags': 'flickr, geospatial',
    'snippet': 'Flickr photos with location information',
    'description': 'Flickr photos with location information'
}
published_item = gis.content.add(item_properties, data=output_geojson)

#publish the GeoJSON as a hosted feature layer
feature_layer = published_item.publish()

#make the feature layer public
feature_layer.share(everyone=True)
print("Feature Layer URL:", feature_layer.url)

Feature Layer URL: https://services8.arcgis.com/jgG4uf0iwjBwCKCg/arcgis/rest/services/FracTracker_Flickr_Map_3/FeatureServer


In [25]:
df = pd.read_csv('attributes_updated.csv')
df.head()

,PhotoID,Title,Description,URL,Latitude,Longitude,AlbumID,AlbumTitle,Identifier,MissionDescription,ProjectType,Partner,County,State,photo_src
0,51604730510,TAuch_Infrastructure-Rover_Pipeline_Repair-Ene...,"Photo citation: Ted Auch, FracTracker Alliance...",https://www.flickr.com/photos/fractracker/5160...,39.868655,-80.833423,72157715916543893,Appalachian Buildout,TAuch,Infrastructure-Rover,Rover_Pipeline_Repair,EnergyTransferPartner,BelmontCounty,OH,https://live.staticflickr.com/65535/5160473051...
1,51604505439,TAuch_Infrastructure-Rover_Pipeline_Repair-Ene...,"Photo citation: Ted Auch, FracTracker Alliance...",https://www.flickr.com/photos/fractracker/5160...,39.868177,-80.832106,72157715916543893,Appalachian Buildout,TAuch,Infrastructure-Rover,Rover_Pipeline_Repair,EnergyTransferPartner,BelmontCounty,OH,https://live.staticflickr.com/65535/5160450543...
2,51603848491,TAuch_Infrastructure-Rover_Pipeline_Repair-Ene...,"Photo citation: Ted Auch, FracTracker Alliance...",https://www.flickr.com/photos/fractracker/5160...,39.868111,-80.832081,72157715916543893,Appalachian Buildout,TAuch,Infrastructure-Rover,Rover_Pipeline_Repair,EnergyTransferPartner,BelmontCounty,OH,https://live.staticflickr.com/65535/5160384849...
3,51603848526,TAuch_Infrastructure-Rover_Pipeline_Repair-Ene...,"Photo citation: Ted Auch, FracTracker Alliance...",https://www.flickr.com/photos/fractracker/5160...,39.868288,-80.830231,72157715916543893,Appalachian Buildout,TAuch,Infrastructure-Rover,Rover_Pipeline_Repair,EnergyTransferPartner,BelmontCounty,OH,https://live.staticflickr.com/65535/5160384852...
4,51604730665,TAuch_Infrastructure-Rover_Pipeline_Repair-Ene...,"Photo citation: Ted Auch, FracTracker Alliance...",https://www.flickr.com/photos/fractracker/5160...,39.864536,-80.832995,72157715916543893,Appalachian Buildout,TAuch,Infrastructure-Rover,Rover_Pipeline_Repair,EnergyTransferPartner,BelmontCounty,OH,https://live.staticflickr.com/65535/5160473066...
